<a href="https://colab.research.google.com/github/fernanda-mv/AnalisisdeDatosUQ/blob/main/Taller_GEIH_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Vamos a importar las librerias necesarias para el analisi de la GEIH
import pandas as pd
import numpy as np

In [2]:
#leyendo los datos de la GEIH 
df_cgsse= pd.read_csv( "https://raw.githubusercontent.com/niconomist98/DataAnalyticsUQ/refs/heads/main/Datos/GEIH2025/cgsse.CSV"
                      ,encoding="latin",sep=";")

df_desocupados= pd.read_csv( "https://raw.githubusercontent.com/niconomist98/DataAnalyticsUQ/refs/heads/main/Datos/GEIH2025/No%20ocupados.CSV"
                      ,encoding="latin",sep=";")    

df_ocupados= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
                      ,encoding="latin",sep=";")

df_ft= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"
                      ,encoding="latin",sep=";")                

/tmp/ipykernel_2051/3859890490.py:8: DtypeWarning: Columns (28,89,133,147,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ocupados= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
/tmp/ipykernel_2051/3859890490.py:11: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ft= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"


In [3]:
# 1. Calculando la poblacion total de colombia: 
ptc = df_cgsse["FEX_C18"].sum()
print('Población total de colombia:', int(ptc))

Población total de colombia: 52045001


In [4]:
#Poblacion en edad de trabajar 

sin_edad_trabajar = df_cgsse[df_cgsse['P6040']<15]['FEX_C18'].sum()
edad_trabajar = df_cgsse[df_cgsse['P6040']>=15]['FEX_C18'].sum() #PET


print('La poblacion total sin edad para trabajar es:',int(sin_edad_trabajar))
print('La poblacion total con edad para trabjar es:',int(edad_trabajar))

La poblacion total sin edad para trabajar es: 11391585
La poblacion total con edad para trabjar es: 40653416


In [5]:
#Calculado ahora la poblacion economicamente activa y definimos como poblacion activamente economica
#Tenemos que usar el DSI dado que en el modulo de no ocupados contiene informacion de personas que no estan buscando trabajo 
#No ocupados para este analisis son los que estan buscando trabajo activamente DSI=1 
pea = df_ocupados["FEX_C18"].sum() + df_desocupados[df_desocupados['DSI']==1]["FEX_C18"].sum()

#Tambien podriamos usar la variable FT=1 que es la fuerza de trabajo que cumple con las condiciones 
#Tiene trabajo o esta buscando trabajo

#definimos la poblacion economicamente inactiva como
#Usamos la columna FFT que son las personas fuera de la fuerza de trabajo 
pei = df_ft[df_ft['FFT']==1]['FEX_C18'].sum()

print(f'La poblacion economicamente activa es:', int(pea))
print(f'La poblacion economicamente inactiva es:', int(pei))

La poblacion economicamente activa es: 25974846
La poblacion economicamente inactiva es: 14678569


In [6]:
# Ahora vamos a calcular la tasa de ocupacion (TO)
TO = (df_ocupados['FEX_C18'].sum() / edad_trabajar)

print(f'La tasa de ocupacion: {TO:.2%}')

#Calculando el desempleo (TD) tenemos que

TD = (df_desocupados[df_desocupados['DSI']==1]['FEX_C18'].sum() / pea)
print(f'La tasa de desempleo es {TD:.2%}')

La tasa de ocupacion: 58.43%
La tasa de desempleo es 8.55%


In [7]:
#Calcule las tasas de desempleo para hombres y mujeres en todos los departamentos de colombia,
#cuales son los departamentos con mayores y menores tasas de desempleo ? 
# Hacemos merge para tener el sexo en el dataset de desocupados
df_desocupados_gen = pd.merge(
    df_desocupados[df_desocupados["DSI"]==1], 
    df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], 
    how='left'
)
# Agrupamos por departamento y sexo
desocupados_sexo = (
    df_desocupados_gen.groupby(['DPTO','P3271'])['FEX_C18']
    .sum()
    .reset_index()
    .rename(columns={'FEX_C18':'desocupados'})
)

# Ocupados también con sexo
df_ocupados_gen = pd.merge(
    df_ocupados, 
    df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], 
    how='left'
)

ocupados_sexo = (
    df_ocupados_gen.groupby(['DPTO','P3271'])['FEX_C18']
    .sum()
    .reset_index()
    .rename(columns={'FEX_C18':'ocupados'})
)

# Unimos ocupados y desocupados para calcular la PEA
pea_sexo = pd.merge(ocupados_sexo, desocupados_sexo, on=['DPTO','P3271'], how='outer').fillna(0)
pea_sexo['PEA'] = pea_sexo['ocupados'] + pea_sexo['desocupados']

# Calculamos la tasa de desempleo (TD)
pea_sexo['TD'] = pea_sexo['desocupados'] / pea_sexo['PEA'] * 100

# Reemplazamos sexo por etiquetas
pea_sexo['Sexo'] = pea_sexo['P3271'].map({1:'Hombres', 2:'Mujeres'})

# Mostramos la tabla final
print(pea_sexo[['DPTO','Sexo','TD']].head(20))

    DPTO     Sexo         TD
0      5  Hombres   4.886691
1      5  Mujeres   9.770415
2      8  Hombres   6.301949
3      8  Mujeres  10.232567
4     11  Hombres   9.821256
5     11  Mujeres   7.078481
6     13  Hombres   4.791098
7     13  Mujeres  12.028807
8     15  Hombres   6.967972
9     15  Mujeres   4.008697
10    17  Hombres   5.583183
11    17  Mujeres   7.677962
12    18  Hombres   2.676220
13    18  Mujeres  15.726909
14    19  Hombres   3.390661
15    19  Mujeres   5.146277
16    20  Hombres   5.991357
17    20  Mujeres  13.209651
18    23  Hombres   5.832048
19    23  Mujeres  19.837285


In [8]:
# Mayor y menor TD por sexo
td_max = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmax()]
td_min = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmin()]

print("Departamentos con MAYOR tasa de desempleo por sexo:")
print(td_max[['DPTO','Sexo','TD']])

print("\nDepartamentos con MENOR tasa de desempleo por sexo:")
print(td_min[['DPTO','Sexo','TD']])

Departamentos con MAYOR tasa de desempleo por sexo:
    DPTO     Sexo         TD
48    81  Hombres  18.057931
49    81  Mujeres  31.660892

Departamentos con MENOR tasa de desempleo por sexo:
    DPTO     Sexo        TD
56    91  Hombres  1.979606
57    91  Mujeres  1.874288


In [9]:
# podemos ver que el departamento con mayor desempleo lo que va en el 2025 es el 81 que corresponde a Arauca
# y con menor desempleo es 91 que es Amazonas 

In [13]:
#Como podemos ver esta columna de genero no existe, entonces hacemos un merge para agregar nuestras columnas
#Entonces vamos a separar entre hombres y mujeres 
#Vamos a calcular la peam y peah

df_desocupados = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==63) | (df_desocupados['DPTO']==66) (df_desocupados['DPTO']==17)]
PQRC = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==63) | (df_desocupados['DPTO']==66) |(df_desocupados['DPTO']==17)]['FEX_C18'].sum()
print(f' POBLACION DEL QUINDIO, RISARALDA y CALDAS:',int(PQRC))

TypeError: 'Series' object is not callable

In [14]:
#Ahora vamos a separarlos por genero

df_desocupados_genero_ec = pd.merge(df_desocupados_ec, df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']],
                       on=['DIRECTORIO','SECUENCIA_P','ORDEN'], how='left')

PHDO = df_desocupados_genero_ec[df_desocupados_genero_ec['P3271']==1]['FEX_C18'].sum()
PMDO = df_desocupados_genero_ec[df_desocupados_genero_ec['P3271']==2]['FEX_C18'].sum()
print(f' Hombres desempleados:',int(PHDO))
print(f' Mujeres desempleados:',int(PMDO))

NameError: name 'df_desocupados_ec' is not defined

In [15]:
#Tomando en consideración únicamente los siguientes grupos de edad 
# vamos ara cada uno de los departamentos, cual es el grupo de edad con mayor y menor tasa de desempleo ? 

def clasificar_edad(x):
    if 18 <= x < 24:
        return "18-24"
    elif 24 <= x < 30:
        return "24-30"
    elif 30 <= x < 40:
        return "30-40"
    elif 40 <= x < 50:
        return "40-50"
    elif x >= 60:
        return "60+"
    else:
        return "Otros"  # Para menores de 18 y personas entre 50-59 que no entran

In [16]:
#Despues armamos la PEA por departamento y grupo de edad
#Hacemos merge para traer la edad (P6040) tanto en ocupados como en desocupados.
#Agrupamos por DPTO y grupo_edad.
#Calculamos TD como antes
# Crear columna grupo de edad en df_cgsse
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)

# --- Desocupados con edad ---
df_desocupados_gen = pd.merge(
    df_desocupados[df_desocupados["DSI"]==1],
    df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],
    how="left"
)

desocupados_edad = (
    df_desocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"]
    .sum()
    .reset_index()
    .rename(columns={"FEX_C18":"desocupados"})
)

# --- Ocupados con edad ---
df_ocupados_gen = pd.merge(
    df_ocupados,
    df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],
    how="left"
)

ocupados_edad = (
    df_ocupados_gen.groupby(["DPTO","grupo_edad"])["FEX_C18"]
    .sum()
    .reset_index()
    .rename(columns={"FEX_C18":"ocupados"})
)

# --- PEA y TD ---
pea_edad = pd.merge(ocupados_edad, desocupados_edad, on=["DPTO","grupo_edad"], how="outer").fillna(0)
pea_edad["PEA"] = pea_edad["ocupados"] + pea_edad["desocupados"]
pea_edad["TD"] = pea_edad["desocupados"] / pea_edad["PEA"] * 100


In [17]:
# Filtramos solo los grupos válidos
pea_edad = pea_edad[pea_edad["grupo_edad"].isin(["18-24","24-30","30-40","40-50","60+"])] 

# Para cada departamento, el grupo de mayor y menor TD
mayores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmax()]
menores = pea_edad.loc[pea_edad.groupby("DPTO")["TD"].idxmin()]

print("Grupo con MAYOR tasa de desempleo por departamento:")
print(mayores[["DPTO","grupo_edad","TD"]])

print("\nGrupo con MENOR tasa de desempleo por departamento:")
print(menores[["DPTO","grupo_edad","TD"]])


Grupo con MAYOR tasa de desempleo por departamento:
     DPTO grupo_edad         TD
0       5      18-24  17.685962
6       8      18-24  22.406918
12     11      18-24  17.797990
18     13      18-24  29.157386
25     15      24-30  14.491221
30     17      18-24   9.972584
36     18      18-24  21.686963
43     19      24-30   7.775308
48     20      18-24  19.042303
54     23      18-24  31.660884
60     25      18-24  15.618810
66     27      18-24  21.102675
72     41      18-24  14.602874
79     44      24-30  24.685637
84     47      18-24  20.675536
90     50      18-24  13.483416
96     52      18-24  16.529172
102    54      18-24  28.074930
109    63      24-30  14.237924
114    66      18-24  15.173556
120    68      18-24  23.222696
127    70      24-30  22.884184
132    73      18-24  19.348020
138    76      18-24  17.821703
144    81      18-24  61.095429
150    85      18-24  27.966955
156    86      18-24  44.878068
163    88      24-30  23.449477
171    91      40-50

In [ ]:
#En la mayoría de los departamentos, el grupo con mayor tasa de desempleo corresponde a los jóvenes de 18 a 24 años,
#lo que refleja las dificultades de inserción laboral de esta población. En algunos casos, como Norte de Santander, 
#Valle del Cauca o Bolívar, el grupo más afectado es el de 24 a 30 años, mostrando que la vulnerabilidad también alcanza 
#a los adultos jóvenes. Existen excepciones aisladas en otros rangos de edad, pero en general el desempleo juvenil es el 
#principal reto en el mercado laboral regional.

In [19]:
# Cuáles son los 3 departamentos con menor tasa de desempleo de mujeres entre los 18 y los 25 años ? 
# Filtrar mujeres de 18 a 24 años
# Paso 1: crear grupos de edad
df_cgsse["grupo_edad"] = df_cgsse["P6040"].apply(clasificar_edad)

# Paso 2: merge de ocupados y desocupados con info de sexo y edad
df_ocupados_gen = pd.merge(
    df_ocupados,
    df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],
    how="left"
)

df_desocupados_gen = pd.merge(
    df_desocupados[df_desocupados["DSI"]==1],
    df_cgsse[["DIRECTORIO","SECUENCIA_P","ORDEN","P3271","grupo_edad"]],
    on=["DIRECTORIO","SECUENCIA_P","ORDEN"],
    how="left"
)

# Paso 3: filtrar mujeres 18-24
df_mujeres_oc = df_ocupados_gen[(df_ocupados_gen["P3271"]==2) & (df_ocupados_gen["grupo_edad"]=="18-24")]
df_mujeres_des = df_desocupados_gen[(df_desocupados_gen["P3271"]==2) & (df_desocupados_gen["grupo_edad"]=="18-24")]

# Paso 4: agrupar por departamento
ocupadas = df_mujeres_oc.groupby("DPTO")["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"ocupadas"})
desocupadas = df_mujeres_des.groupby("DPTO")["FEX_C18"].sum().reset_index().rename(columns={"FEX_C18":"desocupadas"})

# Paso 5: calcular PEA y TD
pea = pd.merge(ocupadas, desocupadas, on="DPTO", how="outer").fillna(0)
pea["PEA"] = pea["ocupadas"] + pea["desocupadas"]
pea["TD"] = pea["desocupadas"] / pea["PEA"] * 100

# Paso 6: ordenar y mostrar los 3 con menor tasa
td_mujeres_18_24_min = pea.sort_values("TD").head(3)
print(td_mujeres_18_24_min)


    DPTO     ocupadas  desocupadas          PEA        TD
31    97   111.385759     0.000000   111.385759  0.000000
28    91   251.281733     0.000000   251.281733  0.000000
18    63  7322.443307   310.224229  7632.667536  4.064427


In [ ]:
# Los tres departamentos con menor tasa de desempleo de mujeres entre 18 y 24 años son Vaupés (0%),
# Amazonas (0%) y Quindío (4,06%), lo que evidencia que en estas regiones las jóvenes mujeres 
# enfrentan menores barreras de acceso al mercado laboral.